In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()



import pystan

In [2]:
log_dir = 'log/fabian_sim/20210824_185803_sim_m9//'
data = load_obj('data', log_dir)


## Run MCMC

In [3]:

model_num = 9

sm = load_obj('sm', 'log/compiled_models/model'+str(model_num)+'/')

param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
stan_names = model_phonebook(model_num)['stan_names']

fit_run = run_mcmc(
    data.get_stan_data(),
    sm,
    num_samples = 1000,
    num_warmup = 1000,
    adapt_engaged=True,
    num_chains = 1,
    log_dir = './log/20210725_224158_debug/'
)
ps = fit_run.extract(permuted=False, pars=param_names)


## Run IBIS

In [4]:
# load existing results

ibis = load_obj('particles', log_dir)


In [5]:
gen_model = 0
model_num = 1


for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(ibis.particles[name])
    w = exp_and_normalise(ibis.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))



Estimate
[-0.01  0.01 -0.07 -0.02  0.03  0.02]


Estimate
[[1.1  0.6  0.54 0.01 0.24 0.21]
 [0.6  1.01 0.42 0.26 0.36 0.36]
 [0.54 0.42 0.92 0.06 0.2  0.19]
 [0.01 0.26 0.06 1.05 0.57 0.62]
 [0.24 0.36 0.2  0.57 0.98 0.52]
 [0.21 0.36 0.19 0.62 0.52 0.94]]


## Plot Both results

In [6]:
# have to resample particles to get rid of weights
ibis.resample_particles()
particles = ibis.particles


In [7]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'mcmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

In [8]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'hmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))


alt.Chart(...)